# Assinatura Digital

#### Alunos: Gabriel Schneider e Luciane Tedesco 

#### Questão 1

- Crie dois pares de chaves (par de chaves A e par de chaves B) e armazene-os em disco.

In [9]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa

# Gerando os pares de chave
private_key_A = rsa.generate(2048)
public_key_A = private_key.publickey()

private_key_B = rsa.generate(2048)
public_key_B = private_key.publickey()

# salvando arquivos
f = open('publicA.pem', 'wb')
f.write(key.publickey().exportKey('PEM'))
f.close()

f = open('privateA.pem', 'wb')
f.write(key.exportKey('PEM'))
f.close()

f = open('publicB.pem', 'wb')
f.write(key.publickey().exportKey('PEM'))
f.close()

f = open('privateB.pem', 'wb')
f.write(key.exportKey('PEM'))
f.close()

AttributeError: module 'cryptography.hazmat.primitives.asymmetric.rsa' has no attribute 'generate'

In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

private_key_A = rsa.generate_private_key(public_exponent=65537,key_size=2048, backend=default_backend())
public_key_A = private_key_A.public_key()

private_key_B = rsa.generate_private_key(public_exponent=65537,key_size=2048, backend=default_backend())
public_key_B = private_key_B.public_key()

# salvando arquivos
f = open('publicaA.pem', 'wb')
f.write(public_key_A.public_bytes(encoding=serialization.Encoding.PEM,format=serialization.PublicFormat.SubjectPublicKeyInfo))
f.close()

f = open('privateA.pem', 'wb')
f.write(private_key_A.private_bytes(encoding=serialization.Encoding.PEM, format=serialization.PrivateFormat.PKCS8, encryption_algorithm=serialization.BestAvailableEncryption(b'mypassword')))
f.close()

f = open('publicaB.pem', 'wb')
f.write(public_key_B.public_bytes(encoding=serialization.Encoding.PEM,format=serialization.PublicFormat.SubjectPublicKeyInfo))
f.close()

f = open('privateB.pem', 'wb')
f.write(private_key_B.private_bytes(encoding=serialization.Encoding.PEM, format=serialization.PrivateFormat.PKCS8, encryption_algorithm=serialization.BestAvailableEncryption(b'mypassword')))
f.close()

#### Questão 2

- Crie um programa que possibilite que o usuário forneça um documento para assinar. Utilize a chave de A para assinar a mensagem e armazene a assinatura em um arquivo binário.

In [2]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

message = b"A message I want to sign"
signature = private_key_A.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

#### Questão 3

- Crie um programa que valide a origem do arquivo. Experimente utilizar a chave de A para validar que a mensagem é de autoria de A

In [8]:
def verify(publickey, message, signature):
    return publickey.verify(
        signature,
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

verify(public_key_A, message, signature)

#### Questão 4
- Experimente utilizar a chave B para validar a autoria do arquivo. O que acontece?